<a href="https://colab.research.google.com/github/patrialyx/Baseline-Model-on-image-sentiment-analysis/blob/main/BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from string import punctuation
import re

## Train_test split

In [ ]:
ls = []
file_dir = '/content/drive/My Drive/the whole memotion dir/memotion_test_set/Overall_sentiment'
for clas in os.listdir(file_dir):
  for fil in os.listdir(os.path.join(file_dir, clas)):
    fil1 = fil.split('_')[1]
    fil2 = int(fil1.split('.')[0])
    ls.append(fil2)

train_df = pd.read_csv('/content/drive/My Drive/labels.csv', index_col = 0)
train_df.drop(ls, inplace=True)
train_df.ffill(axis=1, inplace=True)
train_df.drop(['text_ocr'], axis = 1, inplace=True)
train_df = train_df[train_df['text_corrected'].notna()]

## Clean train_text

In [ ]:
stop = []
with open('/content/drive/My Drive/stopwords.txt', 'r') as stopwords:
  for line in stopwords:
    line = line.replace('\n', '')
    stop.append(line)

keys = []
with open('/content/drive/My Drive/vocab.txt', 'r') as keywords:
  for line in keywords:
    line = line.replace('\n', '')
    keys.append(line)

train_df['cleaned_text'] = train_df['text_corrected'].apply(lambda x: re.sub('[^ ]+\.[^ ]+','',x))
table = str.maketrans('', '', punctuation)
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item.translate(table) for item in x.split()]))
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item.isalpha()]))
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item.lower() for item in x.split()]))
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item in keys]))
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)> 1 ]))
DF_new_row=train_df.loc[train_df['cleaned_text']=='']
DF_new_row
train_df.drop(DF_new_row.index, axis=0,inplace=True)
train_df

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,cleaned_text
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,look friend play years challenge facebook
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,best less years pm
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,follow follow saw everyone vs pics so mine pm ...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,year challenge sweet edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,year challenge no year challenge year challenge
...,...,...,...,...,...,...,...,...
6986,image_6987.jpg,If you're going on-and-on-and-on about your pr...,not_funny,not_sarcastic,not_offensive,motivational,positive,going problems just head not oils know not lis...
6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral,friday chinese new year full force ill waiting...
6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral,must watch movies characters find one another
6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive,less more talking planning food around hate li...


## Clean eval_text

In [ ]:
eval_df = pd.read_csv('/content/drive/My Drive/trial.csv')
eval_df.ffill(axis=1, inplace=True)
eval_df = eval_df[eval_df['corrected_text'].notna()]

eval_df['cleaned_text'] = eval_df['corrected_text'].apply(lambda x: re.sub('[^ ]+\.[^ ]+','',x))
table = str.maketrans('', '', punctuation)
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item.translate(table) for item in x.split()]))
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item.isalpha()]))
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item.lower() for item in x.split()]))
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if item in keys]))
eval_df['cleaned_text'] = eval_df['cleaned_text'].apply(lambda x: ' '.join([item for item in x.split() if len(item)> 1 ]))
eval_new_row=eval_df.loc[eval_df['cleaned_text']=='']
eval_df.drop(eval_new_row.index, axis=0,inplace=True)
eval_df

train_size = int(len(train_df))
eval_size = int(len(eval_df))
print ("Train size: %d" % train_size)
print ("Test size: %d" % eval_size)

Train size: 5676
Test size: 885


## Tokenise/encode

In [ ]:
train_posts = train_df['cleaned_text']
train_tags = train_df['overall_sentiment']
test_posts = eval_df['cleaned_text']
test_tags = eval_df['Overall_Sentiment']

tokenize = text.Tokenizer()

tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)

y_test_1 = encoder.transform(test_tags)
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test_1, num_classes)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (5676, 1010)
x_test shape: (885, 1010)
y_train shape: (5676, 5)
y_test shape: (885, 5)


## Build model and train model


In [ ]:
batch_size = 32
epochs = 400
# Build the model
model = Sequential()
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

print('checkpointing')
saved_area = '/content/drive/My Drive/text_saved_models/A1.2_BOW_sgd/third run'
my_callbacks = [EarlyStopping(patience=10), ModelCheckpoint(
   filepath=os.path.join(saved_area, 'overall_sentiment_run1_BOW_text_model.{epoch:02d}-{val_loss:.2f}.h5'), monitor='val_loss', verbose=1, save_best_only=True,
   save_weights_only=False, mode='auto'), TensorBoard(log_dir='./logs')]

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    callbacks = my_callbacks,
                    validation_data = (x_test, y_test),
                    epochs=epochs,
                    verbose=2
                    )

checkpointing
Train on 5676 samples, validate on 885 samples
Epoch 1/400
 - 1s - loss: 1.4686 - accuracy: 0.4073 - val_loss: 1.3609 - val_accuracy: 0.4565

Epoch 00001: val_loss improved from inf to 1.36087, saving model to /content/drive/My Drive/text_saved_models/A1.2_BOW_sgd/third run/overall_sentiment_run1_BOW_text_model.01-1.36.h5
Epoch 2/400
 - 1s - loss: 1.3360 - accuracy: 0.4487 - val_loss: 1.3005 - val_accuracy: 0.4565

Epoch 00002: val_loss improved from 1.36087 to 1.30048, saving model to /content/drive/My Drive/text_saved_models/A1.2_BOW_sgd/third run/overall_sentiment_run1_BOW_text_model.02-1.30.h5
Epoch 3/400
 - 1s - loss: 1.3032 - accuracy: 0.4482 - val_loss: 1.2820 - val_accuracy: 0.4565

Epoch 00003: val_loss improved from 1.30048 to 1.28200, saving model to /content/drive/My Drive/text_saved_models/A1.2_BOW_sgd/third run/overall_sentiment_run1_BOW_text_model.03-1.28.h5
Epoch 4/400
 - 1s - loss: 1.2910 - accuracy: 0.4475 - val_loss: 1.2736 - val_accuracy: 0.4565

Epoch

## Evaluate

In [ ]:
print('val_preds')
predictions = model.predict(x_test)
val_preds = np.argmax(predictions, axis=-1)
df1 = pd.DataFrame(val_preds)
csv_filename = 'val_preds.csv'
df1.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(val_preds)

print('val_trues')
val_trues = y_test_1
df2 = pd.DataFrame(val_trues)
csv_filename = 'val_trues.csv'
df2.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(val_trues)

print('confusion matrix')
cm = confusion_matrix(val_trues, val_preds)
df3 = pd.DataFrame(cm)
csv_filename = 'confusion_matrix.csv'
df3.to_csv(os.path.join(saved_area, csv_filename), index= True)
print(cm)

print('saving to csv')

report = pd.DataFrame(classification_report(val_trues, val_preds, output_dict=True)).transpose()
print(report)
#CHANGE
csv_filename = 'T1_memotion_overall_sentiment_run1_BOW.csv'
report.to_csv(os.path.join(saved_area, csv_filename), index= True)
   
history = history.history
to_be_saved = 'Validation accuracy: {acc}, loss: {loss}'.format(
           acc=history['val_accuracy'][-1], loss=history['val_loss'][-1])
print(to_be_saved)
           
print('append to csv file')
with open(os.path.join(saved_area, csv_filename),'a') as fd:
   fd.write(to_be_saved)
   fd.close()
   
###save model
print('saving model')
acc=history['val_accuracy'][-1]
#CHANGE
model.save(os.path.join(saved_area, 'T1_memotion_overall_sentiment_run1_BOW_{}.h5'.format(acc)))

loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))
model.save('T1_BOW_run1_acc:%f.h5'.format(acc*200))


val_preds
[4 1 1 2 1 2 1 1 1 1 2 1 1 2 1 1 4 2 2 2 1 4 0 1 1 2 1 1 1 2 0 1 2 2 1 2 2
 1 4 2 0 2 2 2 4 2 2 1 2 4 2 1 2 1 2 1 4 4 2 4 2 2 4 2 2 1 2 2 0 1 2 2 2 2
 2 4 4 1 2 1 0 2 1 3 2 4 2 2 4 2 2 1 4 4 2 1 4 2 2 2 4 0 1 4 2 2 2 4 2 2 1
 2 1 1 2 2 1 1 1 2 2 4 2 2 2 2 0 2 1 2 2 1 2 1 1 2 1 1 2 2 2 2 2 2 2 1 2 2
 2 4 2 2 4 2 2 1 1 2 2 2 2 4 2 2 4 1 4 1 1 2 2 1 2 2 1 1 2 4 2 1 2 1 0 2 2
 1 2 2 1 1 2 2 2 2 2 2 1 1 1 1 2 4 0 2 2 2 0 2 2 2 2 1 2 1 1 2 4 4 2 2 2 2
 2 2 2 2 1 0 2 2 1 4 1 1 4 4 2 1 0 4 2 2 4 2 4 1 1 4 2 2 2 1 0 1 2 2 4 4 2
 1 2 3 1 2 1 2 1 2 1 4 4 2 1 1 2 1 1 2 4 4 2 4 1 2 2 2 2 2 1 1 2 4 2 1 1 2
 1 2 2 2 4 2 2 1 1 4 2 4 1 2 1 4 4 2 4 1 1 2 1 1 2 2 2 2 3 1 1 1 4 2 2 2 2
 4 2 2 2 4 2 2 2 1 2 2 1 2 2 1 4 2 2 1 2 2 4 1 4 2 1 2 2 1 2 2 1 0 1 2 1 3
 4 1 4 1 2 1 1 2 2 2 1 4 1 2 2 1 1 2 2 2 1 4 4 2 2 2 1 0 0 2 0 1 2 1 1 2 2
 2 2 1 0 4 2 2 1 2 2 2 4 2 4 4 0 1 1 2 1 1 4 2 1 1 2 2 0 1 1 1 1 4 1 4 1 0
 2 2 4 4 2 1 1 4 2 2 1 2 2 4 1 1 4 2 0 1 1 1 1 2 2 4 1 1 1 2 4 1 1 4 4 1 2
 1 1 4 2 1 1 2 